In [6]:
from tempo_detector import TempoDetector
import io_utils
import matplotlib.pyplot as plt

In [41]:
TEST_BUCKETS = [(1.0/64,), (1.0/32,), (1.0/32, 'd'), (1.0/32, 't'), (1.0/16, 'd'), (1.0/16, 't'), (1.0/16,),
                (1.0/8, 'd'), (1.0/8, 't'), (1.0/8,),
                (1.0/4, 'd'), (1.0/4, 't'), (1.0/4,),
                (1.0/2, 'd'), (1.0/2,),
                (1.0,), (2.0,), (3.0,), (4.0,), (5.0,)]

# SIMPLE_BUCKETS = TEST_BUCKETS
SIMPLE_BUCKETS = [(1.0/32,),  (1.0/16, 't'), (1.0/16, 'd'), (1.0/16,),
                (1.0/8, 'd'), (1.0/8, 't'), (1.0/8,),
                (1.0/4, 'd'), (1.0/4, 't'), (1.0/4,),
                (1.0/2, 'd'), (1.0/2,),
                (1.0,)]

In [12]:
# DIRS = ['debussy', 'mozart', 'beeth', 'chopin', 'grieg', 'mendelssohn', 'borodin', 'bach']
DIRS = ['mozart']
OUTPUT_DIR = 'predicted_tempos'
START_TEMPOS = range(40, 200, 10)

In [16]:
def predictTempos(E_time_mat, detector):
    predicted_tempos = []
    for i in range(len(E_time_mat)):
        if i % 5 == 0:
            print i, '/', len(E_time_mat)
        detector.setBuckets(TEST_BUCKETS)
        candidates_with_losses = [detector.predict(E_time_mat[i], t, detector.linearDistanceFunction, \
                            detector.linearDistanceFunctionGradient, lambda x: 1) for t in START_TEMPOS]
        candidates_with_losses = list(set(candidates_with_losses))
        print candidates_with_losses
        candidates = [c for c, l in candidates_with_losses]
        E_pred_mat, sigs = io_utils.generatePredictedTempoMatrix(filenames[i], TEST_BUCKETS, candidates)
        detector.setBuckets(SIMPLE_BUCKETS)
        candidates_bucketed_loss = [(c, detector.computeLoss(c, E_time_mat[i], detector.logDistanceFunction)) \
                                       for c in candidates]
        tempo_options = sorted(candidates_bucketed_loss, key=lambda x: x[1])
        print tempo_options
        selected_tempo = tempo_options[0][0]
        predicted_tempos.append(selected_tempo)
    return predicted_tempos

In [38]:
def compareTempos(predicted_tempos, actual_tempos):
    comparison = []
    predicted_tempos = [int(t) for t in predicted_tempos]
    actual_tempos = [int(t) for t in actual_tempos]
    for i in range(len(predicted_tempos)):
        absolute_diff = abs(predicted_tempos[i] - actual_tempos[i])
        pred_over_actual = predicted_tempos[i] / actual_tempos[i]
        if pred_over_actual > 0:
            scale = pred_over_actual
            scaled_diff = min(abs(predicted_tempos[i] - actual_tempos[i] * scale), \
                              abs(predicted_tempos[i] - actual_tempos[i] * (scale+1)))
        else:
            scale = actual_tempos[i] / predicted_tempos[i]
            scaled_diff = min(abs(actual_tempos[i] - scale * predicted_tempos[i]), \
                              abs(actual_tempos[i] - (scale+1) * predicted_tempos[i]))
        comparison.append((absolute_diff, scale, scaled_diff, predicted_tempos[i], actual_tempos[i]))
    return comparison

In [43]:
E_time_mat, tempos, filenames = io_utils.generateTempoPredictionData(DIRS)
detector = TempoDetector(TEST_BUCKETS)
predicted_tempos = predictTempos(E_time_mat[:5], detector)

[25.0, 50.0, 75.0, 33.33333333333333, 150.0, 66.66666666666666, 100.0, 300.0, 133.33333333333331, 200.0, 600.0, 266.66666666666663, 400.0, 1200.0, 800.0, 1600.0, 3200.0, 4800.0, 6400.0, 8000.0]
0 / 5
[(163, 1254240), (85, 795932), (129, 3497337)]
[(163, 1811.1607723235422), (129, 2426.4752545231345), (85, 3713.7985184509175)]
[(92, 506809), (182, 1863736), (47, 214416), (135, 909056), (68, 261512)]
[(182, 27.19060561763219), (135, 79.5640410584953), (92, 201.14251304146313), (68, 339.13324404731304), (47, 566.3764798157503)]
[(110, 115212), (55, 103717), (216, 239782), (75, 102547), (147, 145651)]
[(216, 238.2181042948112), (147, 412.44793069206804), (110, 590.8298135969596), (75, 880.0300145549487), (55, 1267.729659448235)]
[(140, 1153901), (204, 687583), (105, 314607)]
[(204, 697.1016126625676), (140, 1198.8299226451913), (105, 1694.9221787178797)]
[(156, 2271794), (114, 1068533), (223, 2534114), (81, 719812)]
[(223, 1327.1418565200472), (156, 2173.875430184965), (114, 3111.259406304

In [44]:
comparison = compareTempos(predicted_tempos, tempos)

In [45]:
print comparison

[(4, 1, 4, 163, 159), (136, 3, 2, 182, 46), (70, 1, 70, 216, 146), (70, 1, 64, 204, 134), (73, 1, 73, 223, 150)]


In [49]:
print filenames

['mozart/mz_570_1.mid', 'mozart/mz_570_2.mid', 'mozart/mz_570_3.mid', 'mozart/mz_545_1.mid', 'mozart/mz_332_3.mid', 'mozart/mz_330_1.mid', 'mozart/mz_332_2.mid', 'mozart/mz_545_2.mid', 'mozart/mz_330_2.mid', 'mozart/mz_330_3.mid', 'mozart/mz_332_1.mid', 'mozart/mz_545_3.mid', 'mozart/mz_331_2.mid', 'mozart/mz_333_1.mid', 'mozart/mz_331_3.mid', 'mozart/mz_331_1.mid', 'mozart/mz_333_3.mid', 'mozart/mz_333_2.mid', 'mozart/mz_311_1.mid', 'mozart/mz_311_2.mid', 'mozart/mz_311_3.mid']


In [48]:
for i in range(len(predicted_tempos)):
            io_utils.eventListToMIDI(E_pred_mat[i], SIMPLE_BUCKETS, 400, \
                int(predicted_tempos[i]), filenames[i][:-4] + '_' + str(predicted_tempos[i]) + '.mid', OUTPUT_DIR, timeSig=sigs[i][0], keySig=sigs[i][1])

NameError: name 'E_pred_mat' is not defined